In [1]:
import gymnasium as gym
from spotmicro.spot_gym_env import SpotGymEnv

gym.register("spotmicro_v0", SpotGymEnv)

/Users/alejandroaristizabal/Documents/spot_mini_mini/spotmicro/util/pybullet_data


pybullet build time: Jan 18 2024 15:13:35


In [2]:
# env = gym.make("spotmicro_v0")
# done = False
# 
# obs = env.reset()
# tot_reward = 0
# 
# while not done:
#     action = env.action_space.sample()
#     next_obs, reward, done, info = env.step(action)
#     tot_reward += reward
#     obs = next_obs
# 
# env.close()

In [3]:
import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from torch.distributions import Normal
from gymnasium.wrappers import RecordVideo, RecordEpisodeStatistics, NormalizeObservation, NormalizeReward

device = "cpu"

In [4]:
@torch.no_grad()
def test_agent(env, episode_length, policy, epoch, episodes=10):
  # env = RecordVideo(env, f'videos/epoch{epoch}', episode_trigger=lambda e: True)
  ep_returns = []
  for ep in range(episodes):
    state, _ = env.reset()
    done = False
    ep_ret = 0.0
    step = 0

    while not done and step < episode_length:
      loc, scale = policy(state)
      sample = torch.normal(loc, scale)
      action = torch.tanh(sample).cpu().numpy()
      state, reward, done, *_ = env.step(action)
      ep_ret += reward.item()
      step += 1

    ep_returns.append(ep_ret)

  return sum(ep_returns) / episodes

In [5]:
class GradientPolicy(nn.Module):
  def __init__(self, in_features, out_dims, hidden_size=128):
    super().__init__()
    self.fc1 = nn.Linear(in_features, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc_mu = nn.Linear(hidden_size, out_dims)
    self.fc_std = nn.Linear(hidden_size, out_dims)

  def forward(self, x):
    x = torch.tensor(x).float().to(device)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    loc = self.fc_mu(x)
    loc = torch.tanh(loc)
    scale = self.fc_std(x)
    scale = F.softplus(scale) + 0.001
    return loc, scale

In [6]:
class ValueNet(nn.Module):
  def __init__(self, in_features, hidden_size=128):
    super().__init__()
    self.fc1 = nn.Linear(in_features, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, 1)

  def forward(self, x):
    x = torch.tensor(x).float().to(device)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [7]:
env = gym.make('spotmicro_v0', render=False)
policy = GradientPolicy(16, 12).to(device)
print(test_agent(env, 100, policy, -1, episodes=1))
# display_video(-1)

/Users/alejandroaristizabal/miniforge3/envs/spotty/lib/python3.9/site-packages/gymnasium/envs/registration.py:481: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


argv[0]=
[6, 11, 16, 21]
argv[0]=
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
front_left_leg_link_coverb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
front_right_leg_link_coverb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
rear_left_leg_link_coverb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity 

/Users/alejandroaristizabal/miniforge3/envs/spotty/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/Users/alejandroaristizabal/miniforge3/envs/spotty/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:131: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'list'>
  logger.warn(
/Users/alejandroaristizabal/miniforge3/envs/spotty/lib/python3.9/site-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")
/Users/alejandroaristizabal/miniforge3/envs/spotty/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array d

In [8]:
def create_env(env_name, num_envs=2):
  env_creator = [lambda: gym.make(env_name, render=False) for _ in range(num_envs)]
  env = gym.vector.SyncVectorEnv(env_creator)
  env = NormalizeObservation(env)
  env = NormalizeReward(env)
  return env

In [9]:
env = create_env("spotmicro_v0", 10)
obs, info = env.reset()
print("Num envs:", obs.shape[0], "Obs dimensions:", obs.shape[1])

argv[0]=
argv[0]=
[6, 11, 16, 21]
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
front_left_leg_link_coverb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
front_right_leg_link_coverb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
rear_left_leg_link_coverb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity 

/Users/alejandroaristizabal/miniforge3/envs/spotty/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.single_observation_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_observation_space` for environment variables or `env.get_wrapper_attr('single_observation_space')` that will search the reminding wrappers.
  logger.warn(


In [10]:
print("Num envs:", obs.shape[0], "Obs dimensions:", obs.shape[1])

Num envs: 10 Obs dimensions: 16


In [11]:
env.action_space

Box(-1.0, 1.0, (10, 12), float32)

In [12]:
next_obs, reward, done, truncated, info = env.step(env.action_space.sample())

In [13]:
next_obs

array([[-1.58142025, -2.5433006 , -1.62116973, -2.64776507, -2.65679101,
         2.64890427, -1.62630164, -2.09331176,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.31323989, -1.26660655,  0.3166216 , -1.31616885,  2.75068315,
         1.18400124,  0.31720541,  0.13103549,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-1.60455658,  2.43107026, -1.65505241,  2.52452177,  0.24679912,
        -2.7792914 , -1.62281809,  0.42159682,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.14797564, -0.55392335,  0.15309566, -0.57479135, -0.42875213,
         0.4896605 ,  0.11619072,  0.41435713,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.05665779, -0.08567241,  0

In [14]:
import random

class RLDataset(IterableDataset):
  def __init__(self, env, policy, value_net, samples_per_epoch, gamma, lamb, repeats):
    self.env = env
    self.policy = policy
    self.value_net = value_net
    self.samples_per_epoch = samples_per_epoch
    self.gamma = gamma
    self.lamb = lamb
    self.repeats = repeats
    self.obs, _ = self.env.reset()

  @torch.no_grad()
  def __iter__(self):
    transitions = []

    for step in range(self.samples_per_epoch):
      loc, scale = self.policy(self.obs)
      action = torch.normal(loc, scale)
      next_obs, reward, done, truncated, info = self.env.step(action.cpu().numpy())
      experience = [self.obs, loc, scale, action, reward, done, next_obs]
      experience = map(torch.tensor, experience)
      experience = [x.to(device) for x in experience]
      transitions.append(tuple(experience))
      self.obs = next_obs

    transitions = map(torch.stack, zip(*transitions))
    obs_b, loc_b, scale_b, action_b, reward_b, done_b, next_obs_b = transitions
    reward_b = reward_b.unsqueeze(dim=-1)
    done_b = done_b.unsqueeze(dim=-1).float()

    value_b = self.value_net(obs_b)
    next_value_b = self.value_net(next_obs_b)

    td_error_b = reward_b + (1 - done_b) * self.gamma * next_value_b - value_b

    running_gae = torch.zeros((self.env.num_envs, 1), dtype=torch.float32, device=device)
    gae_b = torch.zeros_like(td_error_b)

    for row in range(self.samples_per_epoch - 1, -1, -1):
      running_gae = td_error_b[row] + (1 - done_b[row]) * self.gamma * self.lamb * running_gae
      gae_b[row] = running_gae

    target_b = gae_b + value_b

    num_samples = self.samples_per_epoch * self.env.num_envs
    reshape_fn = lambda x: x.view(num_samples, -1)
    batch = [obs_b, loc_b, scale_b, action_b, reward_b, gae_b, target_b]

    obs_b, loc_b, scale_b, action_b, reward_b, gae_b, target_b = map(reshape_fn, batch)

    for repeat in range(self.repeats):
      idx = list(range(num_samples))
      random.shuffle(idx)

      for i in idx:
        yield obs_b[i], loc_b[i], scale_b[i], action_b[i], reward_b[i], gae_b[i], target_b[i]

In [19]:
from pytorch_lightning import LightningModule, Trainer
import copy

class PPO(LightningModule):
  def __init__(self, env_name, num_envs=2048, batch_size=1024,
               hidden_size=256, samples_per_epoch=10, policy_lr=1e-4,
               value_lr=1e-3, epoch_repeat=4, epsilon=0.3, gamma=0.99,
               lamb=0.95, entropy_coef=0.1, optim=AdamW):
    super().__init__()
    self.env = create_env(env_name, num_envs=num_envs)
    test_env = gym.make(env_name, render=False)
    self.test_env = NormalizeObservation(test_env)
    self.test_env.obs_rms = self.env.obs_rms

    obs_size = self.env.observation_space.shape[1]
    action_dims = self.env.action_space.shape[1]

    self.policy = GradientPolicy(obs_size, action_dims, hidden_size)
    self.value_net = ValueNet(obs_size, hidden_size)
    self.target_value_net = copy.deepcopy(self.value_net)

    self.dataset = RLDataset(self.env, self.policy, self.target_value_net, samples_per_epoch, gamma, lamb, epoch_repeat)
    self.save_hyperparameters()

  def configure_optimizers(self):
    value_opt = self.hparams.optim(self.value_net.parameters(), lr=self.hparams.value_lr)
    policy_opt = self.hparams.optim(self.policy.parameters(), lr=self.hparams.policy_lr)
    return value_opt, policy_opt

  def train_dataloader(self):
    return DataLoader(dataset=self.dataset, batch_size=self.hparams.batch_size)

  def training_step(self, batch, batch_idx, optimizer_idx):
    epsilon = self.hparams.epsilon
    entropy_coef = self.hparams.entropy_coef
    batch = [x.to(device) for x in batch]
    obs_b, loc_b, scale_b, action_b, reward_b, gae_b, target_b = batch

    state_value = self.value_net(obs_b)

    if optimizer_idx == 0:
      # Optimize value network
      loss = F.smooth_l1_loss(state_value, target_b)
      self.log("episode/Value Loss", loss)

    elif optimizer_idx == 1:
      new_loc, new_scale = self.policy(obs_b)
      dist = Normal(new_loc, new_scale)
      log_prob = dist.log_prob(action_b).sum(dim=-1, keepdim=True)

      prev_dist = Normal(loc_b, scale_b)
      prev_log_prob = dist.log_prob(action_b).sum(dim=-1, keepdim=True)

      rho = torch.exp(log_prob - prev_log_prob)

      surrogate_1 = rho * gae_b
      surrogate_2 = rho.clip(1 - epsilon, 1 + epsilon) * gae_b
      policy_loss = - torch.minimum(surrogate_1, surrogate_2)

      entropy = dist.entropy().sum(dim=-1, keepdim=True)
      loss = policy_loss - entropy_coef * entropy

      self.log("episode/Policy Loss", policy_loss.mean())
      self.log("episode/Entropy", entropy.mean())
      self.log("episode/Reward", reward_b.mean())

      return loss.mean()

  def training_epoch_end(self, training_epoch_outputs):
    self.target_value_net.load_state_dict(self.value_net.state_dict())

    if self.current_epoch % 50 == 0:
      average_return = test_agent(self.test_env, 1000, self.policy, self.current_epoch, 1)
      self.log("episode/Test Average Return", average_return)

In [20]:
# Start tensorboard.
!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

rm: /content/lightning_logs/: No such file or directory
rm: /content/videos/: No such file or directory
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 47042), started 0:01:28 ago. (Use '!kill 47042' to kill it.)

In [22]:
algo = PPO("spotmicro_v0", num_envs=512, value_lr=0.001, policy_lr=0.0001, entropy_coef=1e-10)
trainer = Trainer(max_epochs=1500)

trainer.fit(algo)

/Users/alejandroaristizabal/miniforge3/envs/spotty/lib/python3.9/site-packages/gymnasium/envs/registration.py:481: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


argv[0]=
argv[0]=
[6, 11, 16, 21]
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
front_left_leg_link_coverb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
front_right_leg_link_coverb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
rear_left_leg_link_coverb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity 

: 